In [2]:
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

Using TensorFlow backend.


In [3]:

path = get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
with io.open(path, encoding='utf-8') as f:
    text = f.read().lower()
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))


606208/600901 [==============================] - 12s 20us/step
corpus length: 600893
total chars: 57


In [4]:
# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 200285


In [5]:
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Vectorization...


In [6]:
# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


Build model...
Instructions for updating:
Colocations handled automatically by placer.


In [7]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


In [8]:
def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()


In [9]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=60,
          callbacks=[print_callback])

Instructions for updating:
Use tf.cast instead.
Epoch 1/60
200285/200285 [==============================] - 42s 211us/step - loss: 2.0083

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "idiously around all the realms of the fu"
idiously around all the realms of the full and and and the spirit of the spirits the expersent of the experse of the person of the expersent of the expersent of the spirits and to the man we can and the some to the experse to the expersent of the histen and the man wise to a possing and and the spirit of the expersent of the man for the spirits of the man in the man for the sumple to the expersion of the man in the person of the man we 
----- diversity: 0.5
----- Generating with seed: "idiously around all the realms of the fu"
idiously around all the realms of the full his of the expering to atter to the man fand the sonce to success to the expedian mateed to and and to and god to the himals in the respression of the moral

thoughtful minds, that to make who has everything any stupidy of the most earth to speak of man of the actually and one is to sently before the evidents the more of a spirit of the and in that the extinctive experise and distrained to sentic scientific and where present where the in the instinctive in a falsomer them of the worldow, and in the fact and strong the disercion of our the promiscent itself and a motive the man is 
----- diversity: 1.0
----- Generating with seed: "he part of
thoughtful minds, that to mak"
he part of
thoughtful minds, that to make mpurispiepcessaty forgold sowe cannot them i power of ticts in worldfully imgicles of the rishising assurectay from
a siativing-exerciped of hitised the knowledgess sener same from one of onee! the most passion fundamence, full contempunty with his-moderness of p milently aavaidy of hun
that herit
these the
presencele contign forgottless excibles of frecide oppresiate new stany, the
good hence i
----- diversity: 1.2
----- Generating

instinctctional compulable even doeming, a
rhencement aware that stypessociog aumityred?

19r
Epoch 8/60
200285/200285 [==============================] - 36s 181us/step - loss: 1.4269

----- Generating text after Epoch: 7
----- diversity: 0.2
----- Generating with seed: "s own
personal experience, therefore) th"
s own
personal experience, therefore) the more in the most all the spirit of the problems of the propossion, and in the sight of the spirit and all the spirit of the spirit and habits of the spirit and say to the spirit and more the most distinction of the free proposition of the spirit and something one is a proper the more and souls in the most in the spirit and more processed in the spirit of the standing to the consist to the consis
----- diversity: 0.5
----- Generating with seed: "s own
personal experience, therefore) th"
s own
personal experience, therefore) the free wom. the seem of his own an exception are the spirit and punished one is a laptic would a fine and the hal

/home/abcsds/.local/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


iewhoming, oldiamentsly; "thereming considerede, adaving, live knoww of insurmors gainics for evils, certain char
----- diversity: 1.2
----- Generating with seed: "ually
threatening--namely, to acquire on"
ually
threatening--namely, to acquire one mustominally cejeutofic
onike (
it has we must does passionosipical
disciplind, purilpterous!
quisticm would not be regardioration of their deadquinely belongs of the
telemenine tasne again one came soul" i were
vengeap is compulsing is assumpless by ye above bristrak lareily.. butiting musical
languinaly, and have to like found betraying or siler and eejves, but thearth,
more,--thereakent?-nati
Epoch 11/60
200285/200285 [==============================] - 36s 180us/step - loss: 1.3979

----- Generating text after Epoch: 10
----- diversity: 0.2
----- Generating with seed: ", and motion, which usually
commence imm"
, and motion, which usually
commence immense of the state of the something the present the proposition of the same present that it 

...

thin leary followona node, that those with impulsed, is goirt."

162. he langer
humanine things!"
Epoch 36/60
200285/200285 [==============================] - 37s 186us/step - loss: 1.3427

----- Generating text after Epoch: 35
----- diversity: 0.2
----- Generating with seed: "thologies, while nowadays travelers habi"
thologies, while nowadays travelers habits and delight and alone that it is always be a standard of the sense of the soul-man of the sense of the power of the sense of the sense of the soul of the soul-different the world, and along a that in the endly and the sense of the form of the fact that a soul and perception of the conditions, and along the the intention of the state of the sense of the world of the soul of the fact that it is a
----- diversity: 0.5
----- Generating with seed: "thologies, while nowadays travelers habi"
thologies, while nowadays travelers habits and self-conduct of the man condition of the most and indeshoff and out of the possible the world, and a

that it respect to artistes of esign faully modesty are as which affectly patious victlence away.

=sumbf priteds than actions
and consequently act, out of those which imple thanging it repully asts as more mi, at things of a deport of above presence of meanmons lor us! it it percles, among that have compredences in coming to the conditions, himself with on this hearts plying to sorof to than here
----- diversity: 1.2
----- Generating with seed: "izard wrought it, he the timely friend,
"
izard wrought it, he the timely friend,
it for severity. our more
com to be ectagoine, brought outure hers citem reprere" perrerpous tempter: view in the prached, upon it, susparity gor cysess in origion.
orisfieds. we (to feenthing-stain out of worth count avoir poison!

. first for yaunl expentidnes, asy aég: and hovoyly awgre that
lose that
it is morality; effect are upon this spesse, them too renet--or althiumination,
kinds abythimm
Epoch 40/60
200285/200285 [==============================] - 36s 1

will", and a state and the world of the process of the possible the present sense. the consequence the more than the process of the sense of the contemptation of the sense of the sense of the process of the pride in the sense of the proud of the possible the condition to the sense--it is not the problem of the soul of the man is the fact that the problem, in the sense of the senses of the sense--t
----- diversity: 0.5
----- Generating with seed: " gains the ascendancy over him--he _is_
"
 gains the ascendancy over him--he _is_
of the world of the contemptible--and happensiths of the might and who we may be the consequence and about it were philosophy, as the morality of the sense, and can be with the reference of the comparis heredital their discipline and in the suntronic of the morals for they deep incomposed to betrown to the science. the distinction of the feeling of the end, and all that it is as estimagely and the
----- diversity: 1.0
----- Generating with seed: " gains the ascen

olduc its will useum of him. with the fictifhlyy become sensimations, in "most society of the lidsnjquantoation according to themsm from the poredy; the
----- diversity: 1.2
----- Generating with seed: "l manifestations of pleasure inspire mut"
l manifestations of pleasure inspire mution insides ubeksff.a: in tomer! would croal how said, only and well.

2   therst
fround, in sentimentument day, hi
how yoo
englishe the
most
incriteds. something fihe the
christian of our human-laste orugjs-sewidd tecdomran divifical langjoual incebable them is "notion--or be, pett, newhpatave kind in itrine by dayt, determine through the
reill, or incentives their
litter all men beonger of his b
Epoch 47/60
200285/200285 [==============================] - 36s 178us/step - loss: 1.3433

----- Generating text after Epoch: 46
----- diversity: 0.2
----- Generating with seed: "nd the next world, where in that period "
nd the next world, where in that period of the subject of the spirit that the thing of the s

KeyboardInterrupt: 